In [1]:
# 导包
import torch
from torchvision import datasets
from torchvision import transforms
import torch.nn as nn 
import torch.optim as optim

#### 数据集

In [2]:
train_data = datasets.MNIST(root="./mnist/data",train=True,transform=transforms.ToTensor(),download=True)
test_data = datasets.MNIST(root="./mnist/data",train=False,transform=transforms.ToTensor(),download=True)

In [3]:
len(train_data), len(test_data)

(60000, 10000)

In [4]:
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False)

#### 网络结构

In [5]:
# 定义 MLP 网络，继承nn.Module
class MLP(nn.Module):
    
    # 初始化方法
    # input_size输入数据的维度    
    # hidden_size 隐藏层的大小
    # num_classes 输出分类的数量
    def __init__(self, input_size, hidden_size, num_classes):
        # 调用父类的初始化方法
        super(MLP, self).__init__()
        # 定义第1个全连接层  
        self.fc1 = nn.Linear(input_size, hidden_size)
        # 定义激活函数
        self.relu = nn.ReLU()
        # 定义第2个全连接层
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        # 定义第3个全连接层
        self.fc3 = nn.Linear(hidden_size, num_classes)
        
    # 定义forward函数
    # x 输入的数据
    def forward(self, x):
        # 第一层运算
        out = self.fc1(x)
        # 将上一步结果送给激活函数
        out = self.relu(out)
        # 将上一步结果送给fc2
        out = self.fc2(out)
        # 同样将结果送给激活函数
        out = self.relu(out)
        # 将上一步结果传递给fc3
        out = self.fc3(out)
        # 返回结果
        return out

In [6]:
# 定义参数    
input_size = 28 * 28  # 输入大小
hidden_size = 512  # 隐藏层大小
num_classes = 10  # 输出大小（类别数） 

# MLP实例
model = MLP(input_size, hidden_size, num_classes)

#### 损失函数和优化器

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
learning_rate = 0.001 # 学习率
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

#### 模型训练

In [9]:
# 训练网络
num_epochs = 10 # 训练轮数
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 将iamges转成向量
        images = images.reshape(-1, 28 * 28)
        # 将数据送到网络中
        outputs = model(images)
        # 计算损失
        loss = criterion(outputs, labels)
        
        # 首先将梯度清零
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        
        if (i + 1) % 300 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [2/10], Step [600/600], Loss: 0.1695
Epoch [4/10], Step [600/600], Loss: 0.0033
Epoch [6/10], Step [600/600], Loss: 0.0282
Epoch [8/10], Step [600/600], Loss: 0.0181
Epoch [10/10], Step [600/600], Loss: 0.0016


#### 测试效果

In [10]:
# 测试网络
with torch.no_grad():
    correct = 0
    total = 0
    # 从 test_loader中循环读取测试数据
    for images, labels in test_loader:
        # 将images转成向量
        images = images.reshape(-1, 28 * 28)
        # 将数据送给网络
        outputs = model(images)
        # 取出最大值对应的索引  即预测值
        _, predicted = torch.max(outputs.data, 1)
        # 累加label数
        total += labels.size(0)
        # 预测值与labels值比对 获取预测正确的数量
        correct += (predicted == labels).sum().item()
    # 打印最终的准确率
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 98.08 %


#### 保存模型

In [11]:
torch.save(model,"mnist_mlp_model.pkl")